# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

## Part 2. Load Simulation Data

Load the simulation results data.

In [2]:
sim_df = pd.read_pickle(
    "../data/simulations/initial_conditions-2024-01-04_11-38-47.pkl.gz"
).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [3]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,block_utilization,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,history_size,space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_base_fee,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_volume,storage_fee_volume,rewards_to_nominators,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,0.0,256,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1,NaN,standard,stochastic,3000000000
14,1,14400.0,13.698630,13.698630,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.000002,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.097069,13.601561,0.000000,0.000000,0.0,0.000000,0.000000,13.698630,0,2342113616000,0.0,96787476.0,13.69863,1.0,12.0,65483445.0,222,14358.0,0.0,0.0,86380.0,0.000000,0.000000,0.0,1,1.302876e+10,standard,stochastic,3000000000
28,2,28800.0,27.392407,27.397260,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.000001,0.0,1.320000e+09,1.680000e+09,0.000788,0.001410,3.428006,23.962202,0.000000,0.004853,0.0,0.000000,0.000000,13.698630,268435456,4655914240000,0.0,59162244.0,13.69863,1.0,3.0,63686796.0,208,14387.0,0.0,0.0,86497.0,0.000004,0.048534,0.0,1,1.186632e+10,standard,stochastic,3000000000
42,3,43200.0,40.968150,41.144425,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.000002,0.0,1.320000e+09,1.680000e+09,0.040840,0.002580,3.198596,37.726133,0.000021,0.176254,0.0,0.000016,0.000006,13.747165,536870912,5722370272000,0.0,22454222.0,13.69863,2.0,0.0,54761010.0,274,14269.0,0.0,0.0,86143.0,0.000002,1.714003,0.0,1,1.072007e+10,standard,stochastic,3000000000
56,4,57600.0,56.220690,56.557056,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.000003,0.0,1.320000e+09,1.680000e+09,0.076549,0.015701,4.508591,51.619848,0.000183,0.336184,0.0,0.000016,0.000167,15.412631,536870912,7265428336000,0.0,121396328.0,13.69863,1.0,7.0,73049754.0,373,14322.0,0.0,0.0,86602.0,0.000008,1.599298,0.0,1,2.958196e+09,standard,stochastic,3000000000


Simulation Runs.

In [4]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,standard,stochastic,92
1,2,standard,stochastic,92
2,3,standard,stochastic,92
3,4,standard,stochastic,92
4,5,standard,stochastic,92


### Coloring Metrics
Use a constant mapping from columns to colors

In [5]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [6]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


In [7]:
fan_chart_quantile_median(sim_df, column='circulating_supply', median_only=False)

:Overlay
   .Area.I                          :Area   [days_passed]   (min,max)
   .Curve.Circulating_Supply_Median :Curve   [days_passed]   (median)

In [38]:
fan_chart_quantile(sim_df, column='circulating_supply', median_only=False)

:Overlay
   .Area.I                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Circulating_Supply_Quantile :Curve   [days_passed]   (median)

### Stocks

The listed stocks are of four types, which are 
1) **Agent Treasuries**, that consists of a) Farmers Balance, b) Operators Balance, c) Users Balance and d) Nominators Balance; 
2) **Agent Pools**, of which there is an single one: the Operator Staking Pool; 
3) **Protocol Treasuries**, which consists of a) Designated Storage Fund and b) Escrow Fund.
4) **Other**, of which there is an single one: Protocol Issuance.

From an aggregated sectorial perspective, the full description of the token dynamics is done by writing the initial state of the stocks and to formally define the flows between them. One **assumption** is as follows:


| Stock | Type | SSC Quantity at time zero | 
| - | - | - | 
| Protocol Issuance | Other | $\text{TotalIssuance} - \sum \text{Stocks}(t=0)$ | 
|Escrow Fund | Protocol Treasuries | 0.0 |
|Designated Storage Fund | Protocol Treasuries | 0.0 |
|Farmers Balance | Agent Treasuries | 0.0 |
|Operators Balance | Agent Treasuries | 0.0 |
|Nominators Balance | Agent Treasuries | 0.0 |
|Users Balance | Agent Treasuries | $10\%$ of $\text{TotalIssuance}$ |
|Operator Staking Pool | Agent Pools | 0.0 |

Source:  
https://hackmd.io/ywJv4YxfQla3DOktqA9zdg?view#Stocks  
Authors:  
Danilo Lessa Bernardineli (BlockScience), September 2023

## SSC Balances Over Time

System Balances

In [9]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']

In [10]:
hover = HoverTool(
    tooltips=[('Days Passed', '$x{0,0}')]
)
colors = [column_colors[c] for c in system_balances]
sim_df.hvplot.area(x='days_passed', y=system_balances, groupby='run', stacked=True, alpha=1, width=1200, height=500, legend='top_right', ylabel='SSC', tools=[hover], ylim=(0,None), title='SSC System Daily Balances Stacked by Run', color=colors)

:DynamicMap   [run]
   :NdOverlay   [Variable]
      :Area   [days_passed]   (value,Baseline)

In [11]:
hv.Overlay([fan_chart_quantile(sim_df, c) for c in system_balances]).opts(title='SSC System Daily Balances Fan Chart Comparison', ylabel='SSC')

:Overlay
   .Area.I                                 :Area   [days_passed]   (0.25,0.75)
   .Curve.Other_Issuance_Balance_Quantile  :Curve   [days_passed]   (median)
   .Area.II                                :Area   [days_passed]   (0.25,0.75)
   .Curve.Reward_Issuance_Balance_Quantile :Curve   [days_passed]   (median)

### Weekly Aggregation

In [12]:
# Create a weekly index
sim_df['weeks_passed'] = sim_df['days_passed'] // 7

# Group by the weekly index and aggregate, then filter out incomplete weeks
weekly_aggregated_df = (
    sim_df.groupby(['run', 'weeks_passed'])
    .filter(lambda x: len(x) == 7)  # Assuming each week should have 7 days
    .groupby(['run', 'weeks_passed'])
    .sum()
)

In [13]:
weekly_aggregated_df.hvplot.bar(x='weeks_passed', y=system_balances, groupby='run', stacked=False, alpha=1, width=1200, height=500, legend='top_right', ylabel='SSC', tools=[hover], ylim=(0,None), title='SSC System Weekly Balances Compared by Run', rot=90, color=colors).opts(multi_level=False)

:DynamicMap   [run]
   :Bars   [weeks_passed,Variable]   (value)

### Agent Treasuries
Consists of a) Farmers Balance, b) Operators Balance, c) Users Balance and d) Nominators Balance

In [14]:
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'user_supply',
    'nominators_balance',
]
colors = [column_colors[c] for c in agent_balances]

In [15]:
weekly_aggregated_df.hvplot.area(x='weeks_passed', y=agent_balances, groupby='run', stacked=True, alpha=0.9, width=1200, height=500, legend='top_left', ylabel='SSC', tools=[hover], ylim=(0,None), title='SSC Agent Weekly Balances Stacked by Run', color=colors)

:DynamicMap   [run]
   :NdOverlay   [Variable]
      :Area   [weeks_passed]   (value,Baseline)

In [16]:
hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in agent_balances]).opts(title='SSC Agent Weekly Balances Fan Chart Comparison', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Farmers_Balance_Quantile    :Curve   [days_passed]   (median)
   .Area.II                           :Area   [days_passed]   (0.25,0.75)
   .Curve.Operators_Balance_Quantile  :Curve   [days_passed]   (median)
   .Area.III                          :Area   [days_passed]   (0.25,0.75)
   .Curve.User_Supply_Quantile        :Curve   [days_passed]   (median)
   .Area.IV                           :Area   [days_passed]   (0.25,0.75)
   .Curve.Nominators_Balance_Quantile :Curve   [days_passed]   (median)

### Agent Pools
There is an single one: the Operator Staking Pool

In [17]:
agent_pool_balances = ['staking_pool_balance']

hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in agent_pool_balances]).opts(title='SSC Agent Pools Weekly Balances', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)

### Protocol Treasuries
Consists of a) Designated Storage Fund and b) Escrow Fund.

In [18]:
protocol_treasury_balances = ['fund_balance']

hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in protocol_treasury_balances]).opts(title='SSC Agent Pools Weekly Balances', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                      :Area   [days_passed]   (0.25,0.75)
   .Curve.Fund_Balance_Quantile :Curve   [days_passed]   (median)

### Other Balances

In [19]:
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
other_balances

['holders_balance', 'burnt_balance']

In [20]:
colors = [column_colors[c] for c in other_balances]
weekly_aggregated_df.hvplot.area(x='weeks_passed', y=other_balances, groupby='run', stacked=True, alpha=0.9, width=1200, height=500, legend='top_left', ylabel='SSC', tools=[hover], ylim=(0,None), title='SSC Agent Weekly Balances Stacked', color=colors)

:DynamicMap   [run]
   :NdOverlay   [Variable]
      :Area   [weeks_passed]   (value,Baseline)

In [21]:
hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in other_balances]).opts(title='SSC Other Weekly Balances', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                         :Area   [days_passed]   (0.25,0.75)
   .Curve.Holders_Balance_Quantile :Curve   [days_passed]   (median)
   .Area.II                        :Area   [days_passed]   (0.25,0.75)
   .Curve.Burnt_Balance_Quantile   :Curve   [days_passed]   (median)

## SSC Supply Over Time

In [22]:
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
supply_columns

['earned_supply',
 'earned_minus_burned_supply',
 'user_supply',
 'circulating_supply']

In [23]:
hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in supply_columns]).opts(title='SSC Other Weekly Balances', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                                    :Area   [days_passed]   (0.25,0.75)
   .Curve.Earned_Supply_Quantile              :Curve   [days_passed]   (median)
   .Area.II                                   :Area   [days_passed]   (0.25,0.75)
   .Curve.Earned_Minus_Burned_Supply_Quantile :Curve   [days_passed]   (median)
   .Area.III                                  :Area   [days_passed]   (0.25,0.75)
   .Curve.User_Supply_Quantile                :Curve   [days_passed]   (median)
   .Area.IV                                   :Area   [days_passed]   (0.25,0.75)
   .Curve.Circulating_Supply_Quantile         :Curve   [days_passed]   (median)

In [24]:
sim_df.hvplot.area(x='days_passed', y=supply_columns, groupby='run', stacked=True, alpha=0.9, width=1200, height=500, legend='top_right', ylabel='SSC', tools=[hover], ylim=(0,None))

:DynamicMap   [run]
   :NdOverlay   [Variable]
      :Area   [days_passed]   (value,Baseline)

In [25]:
hv.Overlay([fan_chart_quantile(weekly_aggregated_df, c) for c in ['max_credit_supply', 'issued_supply', 'total_supply']]).opts(title='SSC Other Weekly Balances', ylabel='SSC', legend_opts={'location':'bottom_right'})

:Overlay
   .Area.I                           :Area   [days_passed]   (0.25,0.75)
   .Curve.Max_Credit_Supply_Quantile :Curve   [days_passed]   (median)
   .Area.II                          :Area   [days_passed]   (0.25,0.75)
   .Curve.Issued_Supply_Quantile     :Curve   [days_passed]   (median)
   .Area.III                         :Area   [days_passed]   (0.25,0.75)
   .Curve.Total_Supply_Quantile      :Curve   [days_passed]   (median)

### Explore Normalized Numeric Simulation Results

In [26]:
weekly_aggregated_df

days_passed  blocks_passed  circulating_supply  \
run weeks_passed                                                   
1   0                      21       302400.0          296.478649   
    1                      70      1008000.0         1105.350047   
    2                     119      1713600.0         2225.323594   
    3                     168      2419200.0         3314.567788   
    4                     217      3124800.0         4503.658951   
...                       ...            ...                 ...   
30  8                     413      5947200.0         8661.701980   
    9                     462      6652800.0         9672.175495   
    10                    511      7358400.0        10719.510316   
    11                    560      8064000.0        11800.090295   
    12                    609      8769600.0        12879.720378   

                   user_supply  earned_supply  issued_supply  \
run weeks_passed                                               
1   0               298.460259            0.0   1.008000e+10   
    1              1127.576997            0.0   1.176000e+10   
    2              2319.558082            0.0   1.176000e+10   
    3              3506.770794            0.0   1.176000e+10   
    4              4845.390722            0.0   1.176000e+10   
...                        ...            ...            ...   
30  8              9718.492926            0.0   1.176001e+10   
    9             11060.632036            0.0   1.176001e+10   
    10            12402.803965            0.0   1.176001e+10   
    11            13745.087441            0.0   1.176001e+10   
    12            15087.416400            0.0   1.176002e+10   

                  earned_minus_burned_supply  total_supply  sum_of_stocks  \
run weeks_passed                                                            
1   0                                    0.0  1.008000e+10   1.800000e+10   
    1                                    0.0  1.176000e+10   2.100000e+10   
    2                                    0.0  1.176000e+10   2.100000e+10   
    3                                    0.0  1.176000e+10   2.100000e+10   
    4                                    0.0  1.176000e+10   2.100000e+10   
...                                      ...           ...            ...   
30  8                                    0.0  1.176001e+10   2.100000e+10   
    9                                    0.0  1.176001e+10   2.100000e+10   
    10                                   0.0  1.176001e+10   2.100000e+10   
    11                                   0.0  1.176001e+10   2.100000e+10   
    12                                   0.0  1.176002e+10   2.100000e+10   

                  block_utilization  dsf_relative_disbursal_per_day  \
run weeks_passed                                                      
1   0                      0.000011                             0.0   
    1                      0.000015                             0.0   
    2                      0.000010                             0.0   
    3                      0.000011                             0.0   
    4                      0.000012                             0.0   
...                             ...                             ...   
30  8                      0.000012                             0.0   
    9                      0.000014                             0.0   
    10                     0.000012                             0.0   
    11                     0.000010                             0.0   
    12                     0.000011                             0.0   

                  reward_issuance_balance  other_issuance_balance  \
run weeks_passed                                                    
1   0                        9.240000e+09            1.176000e+10   
    1                        9.239999e+09            1.176000e+10   
    2                        9.239998e+09            1.176000e+10   
    3                        

In [27]:
df = weekly_aggregated_df#.set_index('weeks_passed')

# Take numeric columns for normalizationn
df_numeric = df.select_dtypes(include=['number'])

# Create the normalized results
df_normalized = df_numeric / df_numeric.max()

# Add the label column back
df_normalized[['label', 'environmental_label']] = df[['label', 'environmental_label']]

# Drop unecessary columns
df_normalized = df_normalized.drop(['label', 'environmental_label', 'sum_of_stocks', 'buffer_size'], axis=1).fillna(0)
df_normalized.shape

(390, 39)

In [28]:
df_normalized

days_passed  blocks_passed  circulating_supply  user_supply  \
run weeks_passed                                                                
1   0                0.034483       0.034483            0.022075     0.019007   
    1                0.114943       0.114943            0.082300     0.071808   
    2                0.195402       0.195402            0.165689     0.147718   
    3                0.275862       0.275862            0.246790     0.223324   
    4                0.356322       0.356322            0.335325     0.308572   
...                       ...            ...                 ...          ...   
30  8                0.678161       0.678161            0.644917     0.618909   
    9                0.758621       0.758621            0.720153     0.704382   
    10               0.839080       0.839080            0.798133     0.789856   
    11               0.919540       0.919540            0.878589     0.875338   
    12               1.000000       1.000000            0.958974     0.960822   

                  earned_supply  issued_supply  earned_minus_burned_supply  \
run weeks_passed                                                             
1   0                       0.0       0.857142                         0.0   
    1                       0.0       0.999999                         0.0   
    2                       0.0       0.999999                         0.0   
    3                       0.0       0.999999                         0.0   
    4                       0.0       0.999999                         0.0   
...                         ...            ...                         ...   
30  8                       0.0       0.999999                         0.0   
    9                       0.0       1.000000                         0.0   
    10                      0.0       1.000000                         0.0   
    11                      0.0       1.000000                         0.0   
    12                      0.0       1.000000                         0.0   

                  total_supply  block_utilization  \
run weeks_passed                                    
1   0                 0.857142           0.627613   
    1                 0.999999           0.837166   
    2                 0.999999           0.574335   
    3                 0.999999           0.598959   
    4                 0.999999           0.673428   
...                        ...                ...   
30  8                 0.999999           0.653377   
    9                 1.000000           0.786366   
    10                1.000000           0.665818   
    11                1.000000           0.559294   
    12                1.000000           0.592979   

                  dsf_relative_disbursal_per_day  reward_issuance_balance  \
run weeks_passed                                                            
1   0                                        0.0                 1.000000   
    1                                        0.0                 1.000000   
    2                                        0.0                 1.000000   
    3                                        0.0                 1.000000   
    4                                        0.0                 1.000000   
...                                          ...                      ...   
30  8                                        0.0                 0.999999   
    9                                        0.0                 0.999999   
    10                                       0.0                 0.999999   
    11                                       0.0                 0.999999   
    12                                       0.0                 0.999998   

                  other_issuance_balance  operators_balance  \
run weeks_passed                                              
1   0                                1.0           0.000786   
    1                                1.0           0.002522   
    2

### Normalized Weekly Means of All Numeric Columns

In [29]:
hv.Overlay([fan_chart_quantile(df_normalized, c, median_only=True) for c in df_normalized.columns]).opts(title='SSC Other Weekly Balances', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Curve.Days_Passed_Quantile                       :Curve   [days_passed]   (median)
   .Curve.Blocks_Passed_Quantile                     :Curve   [days_passed]   (median)
   .Curve.Circulating_Supply_Quantile                :Curve   [days_passed]   (median)
   .Curve.User_Supply_Quantile                       :Curve   [days_passed]   (median)
   .Curve.Earned_Supply_Quantile                     :Curve   [days_passed]   (median)
   .Curve.Issued_Supply_Quantile                     :Curve   [days_passed]   (median)
   .Curve.Earned_Minus_Burned_Supply_Quantile        :Curve   [days_passed]   (median)
   .Curve.Total_Supply_Quantile                      :Curve   [days_passed]   (median)
   .Curve.Block_Utilization_Quantile                 :Curve   [days_passed]   (median)
   .Curve.Dsf_Relative_Disbursal_Per_Day_Quantile    :Curve   [days_passed]   (median)
   .Curve.Reward_Issuance_Balance_Quantile           :Curve   [days_passed]   (median)
   .Curve.Other_Issuance_Balance_Quantile            :Curve   [days_passed]   (median)
   .Curve.Operators_Balance_Quantile                 :Curve   [days_passed]   (median)
   .Curve.Nominators_Balance_Quantile                :Curve   [days_passed]   (median)
   .Curve.Holders_Balance_Quantile                   :Curve   [days_passed]   (median)
   .Curve.Farmers_Balance_Quantile                   :Curve   [days_passed]   (median)
   .Curve.Staking_Pool_Balance_Quantile              :Curve   [days_passed]   (median)
   .Curve.Fund_Balance_Quantile                      :Curve   [days_passed]   (median)
   .Curve.Burnt_Balance_Quantile                     :Curve   [days_passed]   (median)
   .Curve.Nominator_Pool_Shares_Quantile             :Curve   [days_passed]   (median)
   .Curve.Operator_Pool_Shares_Quantile              :Curve   [days_passed]   (median)
   .Curve.Block_Reward_Quantile                      :Curve   [days_passed]   (median)
   .Curve.History_Size_Quantile                      :Curve   [days_passed]   (median)
   .Curve.Space_Pledged_Quantile                     :Curve   [days_passed]   (median)
   .Curve.Allocated_Tokens_Quantile                  :Curve   [days_passed]   (median)
   .Curve.Reference_Subsidy_Quantile                 :Curve   [days_passed]   (median)
   .Curve.Average_Base_Fee_Quantile                  :Curve   [days_passed]   (median)
   .Curve.Average_Priority_Fee_Quantile              :Curve   [days_passed]   (median)
   .Curve.Average_Compute_Weight_Per_Tx_Quantile     :Curve   [days_passed]   (median)
   .Curve.Average_Transaction_Size_Quantile          :Curve   [days_passed]   (median)
   .Curve.Transaction_Count_Quantile                 :Curve   [days_passed]   (median)
   .Curve.Average_Compute_Weight_Per_Bundle_Quantile :Curve   [days_passed]   (median)
   .Curve.Average_Bundle_Size_Quantile               :Curve   [days_passed]   (median)
   .Curve.Bundle_Count_Quantile                      :Curve   [days_passed]   (median)
   .Curve.Compute_Fee_Volume_Quantile                :Curve   [days_passed]   (median)
   .Curve.Storage_Fee_Volume_Quantile                :Curve   [days_passed]   (median)
   .Curve.Rewards_To_Nominators_Quantile             :Curve   [days_passed]   (median)
   .Curve.Average_Compute_Weight_Per_Budle_Quantile  :Curve   [days_passed]   (median)
   .Curve.Max_Credit_Supply_Quantile                 :Curve   [days_passed]   (median)

### Daily Balances Min Max and Mean

In [30]:
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [31]:
balance_columns

['nominators_balance',
 'fund_balance',
 'holders_balance',
 'operators_balance',
 'farmers_balance',
 'staking_pool_balance',
 'burnt_balance']

In [32]:
hv.Overlay([fan_chart_quantile(sim_df, c, median_only=False) for c in balance_columns]).opts(title='SSC Daily Balances Fan Chart Comparison', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
   .Area.II                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
   .Area.III                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
   .Area.IV                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
   .Area.V                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
   .Area.VI                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
   .Area.VII                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)

What's happening in the following, it looks weird: 

In [33]:
weekly_aggregated_df.hvplot.bar(x='weeks_passed', y=balance_columns, groupby='run', stacked=False, alpha=1, width=1200, height=500, legend='top_right', ylabel='SSC', tools=[hover], ylim=(0,None), title='SSC System Weekly Balances Compared by Run', rot=90, color=colors).opts(multi_level=False)

:DynamicMap   [run]
   :Bars   [weeks_passed,Variable]   (value)

Box Charts

In [107]:
box_df = sim_df.set_index(['weeks_passed', 'run'])[balance_columns]
box_df

nominators_balance  fund_balance  holders_balance  \
weeks_passed run                                                      
0            1              0.000000      0.000000         0.000000   
             1              0.000000      0.000000         0.097069   
             1              0.001410      0.004853         3.428006   
             1              0.002580      0.176254         3.198596   
             1              0.015701      0.336184         4.508591   
...                              ...           ...              ...   
12           30           108.523334    280.871270       529.458963   
             30           106.700858    283.591347       533.581653   
             30           107.320851    286.422802       524.021708   
             30           106.167294    291.242277       474.135142   
13           30           127.125964    295.794872       528.995493   

                  operators_balance  farmers_balance  staking_pool_balance  \
weeks_passed run                                                             
0            1             0.000000         0.000000              0.000000   
             1             0.000000        13.601561              0.000000   
             1             0.000788        23.962202              0.000000   
             1             0.040840        37.726133              0.000021   
             1             0.076549        51.619848              0.000183   
...                             ...              ...                   ...   
12           30           54.571719      1146.608836             35.384749   
             30           68.778277      1151.165340             38.998647   
             30           87.063132      1165.280884             40.103997   
             30           94.678828      1230.129524             41.257554   
13           30           87.045406      1180.530550             45.515581   

                  burnt_balance  
weeks_passed run                 
0            1         0.000000  
             1         0.000000  
             1         0.000000  
             1         0.000000  
             1         0.000000  
...                         ...  
12           30        0.073674  
             30        0.073674  
             30        0.073674  
             30        0.073674  
13           30        0.073674  

[2760 rows x 7 columns]

In [108]:
box_df.describe()

,nominators_balance,fund_balance,holders_balance,operators_balance,farmers_balance,staking_pool_balance,burnt_balance
count,2760.000000,2760.000000,2760.000000,2760.000000,2760.000000,2760.000000,2760.000000
mean,23.841586,121.901548,154.828392,13.834467,764.062009,9.013709,0.016179
std,30.652532,108.424980,169.228700,19.123009,426.256414,12.035156,0.027673
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.453413,19.002948,31.242226,1.658291,384.564392,0.374900,0.000108
50%,11.241708,92.150906,88.814704,5.995273,844.905940,3.420997,0.003042
75%,33.296234,215.790954,221.301365,17.424413,1122.618782,13.360848,0.020530
max,179.982183,371.682098,871.397585,138.418739,1700.697692,66.484222,0.217528


In [109]:
colors = [column_colors[c] for c in balance_columns]

In [110]:
box_df.melt().hvplot.violin(y='value', by='variable', c='variable', legend='top_right', width=1200, height=500, title=f'Agent Balances Violin Chart by Across All Weeks and All Runs', cmap=column_colors, ylim=(0,box_df.max().max()*0.75))

:Violin   [variable]   (value)

In [111]:
box_df.reset_index().drop('weeks_passed',axis=1).melt(id_vars=['run']).hvplot.violin(y='value', by='variable', c='variable', groupby='run', legend='top_right', width=1200, height=500, title=f'Agent Balances Violin Chart Across Weeks by Run', cmap=column_colors, ylim=(0,box_df.max().max()*0.75))

:DynamicMap   [run]
   :Violin   [variable]   (value)

In [112]:
box_df.reset_index().drop('run',axis=1).melt(id_vars=['weeks_passed']).hvplot.violin(y='value', by='variable', c='variable', groupby='weeks_passed', legend='top_right', width=1200, height=500, title=f'Agent Balances Violin Chart Across Runs by Week', cmap=column_colors, ylim=(0,box_df.max().max()*0.75))

:DynamicMap   [weeks_passed]
   :Violin   [variable]   (value)